# Predicting NYC Taxi Fares And Model Explainer

In this quickstart, we will be using a subset of NYC Taxi & Limousine Commission - green taxi trip records available from [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/). The data is enriched with holiday and weather data. We will use data transformations and the GradientBoostingRegressor algorithm from the scikit-learn library to train a regression model to predict taxi fares in New York City based on input features such as, number of passengers, trip distance, datetime, holiday information and weather information.

The primary goal of this quickstart is to explain the predictions made by our trained model with the various [Azure Model Interpretability](https://docs.microsoft.com/en-us/azure/machine-learning/service/machine-learning-interpretability-explainability) packages of the Azure Machine Learning Python SDK.

### Azure Machine Learning and Model Interpretability SDK-specific Imports

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import sklearn
from sklearn.externals import joblib
import math

print("pandas version: {} numpy version: {}".format(pd.__version__, np.__version__))

sklearn_version = sklearn.__version__
print('The scikit-learn version is {}.'.format(sklearn_version))

import azureml
from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model

from interpret.ext.blackbox import TabularExplainer
from azureml.interpret.scoring.scoring_explainer import TreeScoringExplainer, save

print('The azureml.core version is {}.'.format(azureml.core.VERSION))

pandas version: 0.23.4 numpy version: 1.16.2
The scikit-learn version is 0.20.3.
The azureml.core version is 1.0.83.


### Setup
To begin, you will need to provide the following information about your Azure Subscription.

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*).

You can get all of these values from the lab guide on the right:
1. In the tabs at the top of the lab guide, select `Environment Details`.
2. Copy the values from SubscriptionID, ResourceGroup, WorkspaceName and WorkspaceRegion and paste them as the values in the cell below.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [2]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "6aa3be9e-6130-409f-882c-cdc4e4816a36" 

#Provide value for the ResourceGroup 
resource_group = "AI-Practice-Dev" 

#Provide the WorkspaceName and WorkspaceRegion of the Azure Machine Learning Workspace
workspace_name = "aml-ai-practice-zst" 
workspace_region = "eastus" 


experiment_name = "lab-explainability"

### Create and connect to an Azure Machine Learning Workspace

Run the following cell to connect to your existing Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You may be prompted to login in the text that is output below the cell. If you are, be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [3]:
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Workspace configuration succeeded


### Train the Model

Run the following cell to download the dataset, split the data into training and test sets and create a pipeline that includes a few steps to clean and standardize the data and ultimately train the model. 

NOTE: Do not get too concerned about the details of the following code. If you take anything away from this cell, it should be that a model has been trained and stored in the variable `clf`.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error

data_url = ('https://introtomlsampledata.blob.core.windows.net/data/nyc-taxi/nyc-taxi-sample-data.csv')

df = pd.read_csv(data_url)
x_df = df.drop(['totalAmount'], axis=1)
y_df = df['totalAmount']

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=0)

categorical = ['normalizeHolidayName', 'isPaidTimeOff']
numerical = ['vendorID', 'passengerCount', 'tripDistance', 'hour_of_day', 'day_of_week', 
             'day_of_month', 'month_num', 'snowDepth', 'precipTime', 'precipDepth', 'temperature']

numeric_transformations = [([f], Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])) for f in numerical]
    
categorical_transformations = [([f], OneHotEncoder(handle_unknown='ignore', sparse=False)) for f in categorical]

transformations = numeric_transformations + categorical_transformations

clf = Pipeline(steps=[('preprocessor', DataFrameMapper(transformations)),
                      ('regressor', GradientBoostingRegressor())])

clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)
y_actual = y_test.values.flatten().tolist()
rmse = math.sqrt(mean_squared_error(y_actual, y_predict))
print('The RMSE score on test data for GradientBoostingRegressor: ', rmse)

The RMSE score on test data for GradientBoostingRegressor:  4.102557373693153


## Global Explanation Using a Meta Explainer (TabularExplainer)

**Global Model Explanation** is a holistic understanding of how the model makes decisions. It provides you with insights on what features are most important and their relative strengths in making model predictions.

To initialize an explainer object, you need to pass your model and some training data to the explainer's constructor.

*Note that you can pass in your feature transformation pipeline to the explainer to receive explanations in terms of the raw features before the transformation (rather than engineered features).*

In [12]:
# "features" and "classes" fields are optional
trained_gradient_boosting_regressor = clf.steps[-1][1]
tabular_explainer = TabularExplainer(trained_gradient_boosting_regressor, 
                                     initialization_examples=X_train, 
                                     features=X_train.columns,  
                                     transformations=transformations)

[TabularExplainer](https://docs.microsoft.com/en-us/python/api/azureml-explain-model/azureml.explain.model.tabularexplainer?view=azure-ml-py) uses one of three explainers: TreeExplainer, DeepExplainer, or KernelExplainer, and is automatically selecting the most appropriate one for our use case. 

You can learn more about the underlying model explainers at [Azure Model Interpretability](https://docs.microsoft.com/en-us/azure/machine-learning/service/machine-learning-interpretability-explainability).

### Get the global feature importance values

Run the below cell and observe the sorted global feature importance. You will note that `tripDistance` is the most important feature in predicting the taxi fares, followed by `hour_of_day`, and `day_of_week`.

In [14]:
# You can use the training data or the test data here
global_explanation = tabular_explainer.explain_global(X_test)

# Sorted feature importance values and feature names
sorted_global_importance_values = global_explanation.get_ranked_global_values()
sorted_global_importance_names = global_explanation.get_ranked_global_names()
dict(zip(sorted_global_importance_names, sorted_global_importance_values))

{'tripDistance': 6.685573255831272,
 'hour_of_day': 0.44367850317900714,
 'day_of_week': 0.24822281540243285,
 'passengerCount': 0.036345809736741806,
 'precipDepth': 0.031102574085268692,
 'temperature': 0.028880685645032354,
 'day_of_month': 0.024896616555696364,
 'isPaidTimeOff': 0.013719743326259559,
 'snowDepth': 0.011707454686670362,
 'month_num': 0.006763590202535178,
 'normalizeHolidayName': 0.00283472856480408,
 'vendorID': 0.00210316225774988,
 'precipTime': 0.001010658748108876}

## Visualizing the Global Explanation

Run the following cell to create a dashbaord the enables you to explore the data and visualize the explanation.

In the Dashboard that is displayed, try answering the following questions:

1. Select the `Data Exploration` tab, the set the `X value` to `tripDistance` and the `Y value` to `PredictedY` (this is predicted fare mount). What happens to the predicted fare as the trip distance increases? 
2. Select the `Global Importance` tab. Drag the slider under Top K Features so its value is set to `3`. What are the top 3 most important features? Which feature has the highest feature importance (and is therefore the most important feature)?

In [36]:
from interpret_community.widget import ExplanationDashboard

ExplanationDashboard(global_explanation, model=clf, datasetX=X_test)

ExplanationWidget(value={'predictedY': [29.47547328106775, 11.277590683810248, 9.538352154971939, 11.077700453…

## Local Explanation

You can use the [TabularExplainer](https://docs.microsoft.com/en-us/python/api/azureml-explain-model/azureml.explain.model.tabularexplainer?view=azure-ml-py) for a single prediction. You can focus on a single instance and examine model prediction for this input, and explain why.

We will create two sample inputs to explain the individual predictions.

- **Data 1**
 - 4 Passengers at 3:00PM, Friday July 5th, temperature 80F, travelling 10 miles

- **Data 2**
 - 1 Passenger at 6:00AM, Monday January 20th, rainy, temperature 35F, travelling 5 miles

In [15]:
# Create the test dataset
columns = ['vendorID', 'passengerCount', 'tripDistance', 'hour_of_day', 'day_of_week', 'day_of_month', 
           'month_num', 'normalizeHolidayName', 'isPaidTimeOff', 'snowDepth', 'precipTime', 
           'precipDepth', 'temperature']

data = [[1, 4, 10, 15, 4, 5, 7, 'None', False, 0, 0.0, 0.0, 80], 
        [1, 1, 5, 6, 0, 20, 1, 'Martin Luther King, Jr. Day', True, 0, 2.0, 3.0, 35]]

data_df = pd.DataFrame(data, columns = columns)

In [16]:
# explain the test data
local_explanation = tabular_explainer.explain_local(data_df)

# sorted feature importance values and feature names
sorted_local_importance_names = local_explanation.get_ranked_local_names()
sorted_local_importance_values = local_explanation.get_ranked_local_values()

# package the results in a DataFrame for easy viewing
results = pd.DataFrame([sorted_local_importance_names[0][0:5], sorted_local_importance_values[0][0:5], 
                        sorted_local_importance_names[1][0:5], sorted_local_importance_values[1][0:5]], 
                       columns = ['1st', '2nd', '3rd', '4th', '5th'], 
                       index = ['Data 1', '', 'Data 2', ''])
print('Top 5 Local Feature Importance')
results

Top 5 Local Feature Importance


,1st,2nd,3rd,4th,5th
Data 1,tripDistance,hour_of_day,passengerCount,day_of_week,temperature
,23.7364,0.817527,0.405426,0.130634,0.125084
Data 2,tripDistance,temperature,day_of_week,month_num,precipTime
,7.71724,0.088496,0.0766746,0.019932,7.48644e-05


As we saw from the Global Explanation that the **tripDistance** is the most important global feature. Other than `tripDistance`, the rest of the top 5 important features were different for the two samples.

- Data 1: Passenger count 4 and 3:00 PM on Friday were also important features in the prediction.
- Data 2: The weather-related features (rainy, temperature 35F), day of the week (Monday) and month (January) were also important.

## You're Done!
Congratulations you have finished with this lab.